FOR REGRESSION


In [1]:
pip install torch scikit-learn


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_california_housing  # Changed from load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the California housing dataset
housing = fetch_california_housing()
X, y = housing.data, housing.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

In [8]:
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)   # input_size → 16 neurons
        self.fc2 = nn.Linear(16, 8)   # 16 → 8
        self.fc3 = nn.Linear(8, 1)    # 8 → 1 output for regression

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
input_size = X_train_tensor.shape[1] # Get the input size from the training data
model = NeuralNet(input_size)
criterion = nn.MSELoss()  # good for regression
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [10]:
epochs = 100
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train_tensor) # Use the tensor variable
    loss = criterion(outputs, y_train_tensor) # Use the tensor variable

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 2.9750
Epoch [20/100], Loss: 1.1902
Epoch [30/100], Loss: 0.8042
Epoch [40/100], Loss: 0.7135
Epoch [50/100], Loss: 0.6354
Epoch [60/100], Loss: 0.5725
Epoch [70/100], Loss: 0.5284
Epoch [80/100], Loss: 0.4901
Epoch [90/100], Loss: 0.4581
Epoch [100/100], Loss: 0.4307


In [14]:
with torch.no_grad():  # no gradient calculation during evaluation
    predicted = model(X_test_tensor) # Use the tensor variable
    test_loss = criterion(predicted, y_test_tensor) # Calculate test loss using MSELoss
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 0.4424


FOR CLASSIFICATION

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [16]:
# Load dataset
iris = load_iris()
X = iris.data
y = iris.target

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to tensors
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)


In [17]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(4, 16)   # 4 input features → 16 neurons
        self.fc2 = nn.Linear(16, 8)   # 16 → 8
        self.fc3 = nn.Linear(8, 3)    # 8 → 3 output classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [18]:
model = NeuralNet()
criterion = nn.CrossEntropyLoss()  # good for classification
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [19]:
epochs = 100
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.6684
Epoch [20/100], Loss: 0.4299
Epoch [30/100], Loss: 0.2969
Epoch [40/100], Loss: 0.1930
Epoch [50/100], Loss: 0.1051
Epoch [60/100], Loss: 0.0683
Epoch [70/100], Loss: 0.0541
Epoch [80/100], Loss: 0.0483
Epoch [90/100], Loss: 0.0458
Epoch [100/100], Loss: 0.0442


In [20]:
with torch.no_grad():  # no gradient calculation during evaluation
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print(f'Accuracy: {accuracy*100:.2f}%')


Accuracy: 100.00%
